In [13]:
import pandas as pd
import os
import numpy as np
import re

In [29]:
pc=pd.read_csv('C:/Users/ttodd/OneDrive/Desktop/capstone/final data/PCE_ALL_AREAS.csv')

In [30]:
state_list=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware",'District of Columbia',"Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
pc=pc.loc[pc['GeoName'].isin(state_list),:]
pc_percap=pd.read_csv('C:/Users/ttodd/OneDrive/Desktop/capstone/final data/PCPCE_ALL_AREAS.csv')
pc_percap=pc_percap.loc[pc_percap['GeoName'].isin(state_list),:]

In [31]:
cleaned_column=['GeoName','Description','1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015','2016']
pc_percap=pc_percap.loc[:,cleaned_column]
melted_df=pc_percap.melt(id_vars=['Description','GeoName'])

In [77]:
def pivot_pc_df(df):
    grouped_df=df.groupby(['Description'])
    count=0
    for name, group in grouped_df:
        group=group.drop('Description',axis=1)
        group.columns=['GeoName','year','pce'+name]
        if count==0:
            final_df=group
        else:
            final_df=pd.merge(final_df,group,on=['GeoName','year'])
        count=count+1
    return final_df

In [85]:
def pivot_df(df):
    grouped_df=df.groupby(['Description'])
    count=0
    for name, group in grouped_df:
        group=group.drop('Description',axis=1)
        group.columns=['GeoName','year',name]
        if count==0:
            final_df=group
        else:
            final_df=pd.merge(final_df,group,on=['GeoName','year'])
        count=count+1
    return final_df

In [78]:
pce_all_year=pivot_pc_df(melted_df)

In [39]:
final_df.to_csv('pce_all_year.csv',index=False)

In [202]:
#clean personal income data 

In [83]:
variable_list=[]
data_list=['SA1_1929_2017.csv','SA51_1948_2017.csv']
data_path='C:/Users/ttodd/OneDrive/Desktop/capstone/personal_income/spi0318/'
for i in os.listdir(data_path):
    if i.endswith('ALL_AREAS.csv'):
        data_list.append(i)

In [84]:
data_list

['SA1_1929_2017.csv',
 'SA51_1948_2017.csv',
 'SA25N_1998_2016__ALL_AREAS.csv',
 'SA25_1969_2001__ALL_AREAS.csv',
 'SA27N_1998_2016__ALL_AREAS.csv',
 'SA27_1969_2001__ALL_AREAS.csv',
 'SA30_1958_2016__ALL_AREAS.csv',
 'SA35_1929_2016__ALL_AREAS.csv',
 'SA40_1958_2016__ALL_AREAS.csv',
 'SA45_1969_2016__ALL_AREAS.csv',
 'SA4_1929_2017__ALL_AREAS.csv',
 'SA50_1948_2016__ALL_AREAS.csv',
 'SA5H_1929_1957__ALL_AREAS.csv',
 'SA5N_1998_2017__ALL_AREAS.csv',
 'SA5_1958_2001__ALL_AREAS.csv',
 'SA6N_1998_2017__ALL_AREAS.csv',
 'SA6_1958_2001__ALL_AREAS.csv',
 'SA7H_1929_1957__ALL_AREAS.csv',
 'SA7N_1998_2017__ALL_AREAS.csv',
 'SA7_1958_2001__ALL_AREAS.csv']

In [44]:
def clean_pi_df(temp_data): 
    temp_data=temp_data.drop_duplicates()
    temp_data=temp_data.dropna(subset=['GeoName','Description'])
    temp_data.replace(0,np.nan)
    temp_data.replace(0.0,np.nan)
    temp_data.dropna(thresh=2)
    drop_column=temp_data['Description'].value_counts()
    drop_column=list(drop_column[drop_column!=60].index)
    temp_data=temp_data.loc[~temp_data['Description'].isin(drop_column),:]
    return temp_data

In [102]:
final_result=pd.DataFrame({'state':state_list})
df_count=0
for i in data_list:
    temp_data=pd.read_csv(data_path+i)
    temp_data_column=temp_data.columns
    temp_data_column=list(temp_data_column)
    final_column_part1=['GeoName','Description']
    final_column_part2=temp_data_column[7:]
    final_column_part2=[x for x in final_column_part2 if x>='1997' and x<='2016']
    final_column=final_column_part1+final_column_part2
    if len(final_column)==22:
        temp_data=temp_data.loc[:,final_column]
        temp_data=clean_pi_df(temp_data)
        temp_melted_df=temp_data.melt(id_vars=['Description','GeoName'])
        temp_pi_df=pivot_df(temp_melted_df)
        temp_column=temp_pi_df.columns
        temp_pi_df['GeoName']=temp_pi_df['GeoName'].str.replace('\*','')
        cleaned_column=[re.sub('([0-9]{1}\/)|(^\s+)','',x) for x in temp_column]
        temp_pi_df.columns=cleaned_column
        #print(temp_pi_df.head(5))
        if df_count==0:
            final_result=temp_pi_df
        else:
            final_result=pd.merge(final_result,temp_pi_df,on=['GeoName','year'],how='inner',suffixes=('', '_y'))    
        df_count+=1

drop_columns=[x for x in final_result.columns if x.endswith('_y')]
final_result=final_result.drop(drop_columns,axis=1)

In [133]:
final_result=final_result.loc[final_result['GeoName'].isin(state_list),:]

In [135]:
final_result.to_csv('pi_all_year.csv',index=False)

In [203]:
#electricity data

In [174]:
elec=pd.read_csv('C:/Users/ttodd/OneDrive/Desktop/capstone/final data/Average_retail_price_of_electricity.csv',skiprows=4)
#units is cents per killowatthour
elec['GeoName'], elec['type'] = elec['description'].str.split(':', 1).str
elec=elec.drop(['units','source key','description'],axis=1)
elec=elec.dropna(subset=['type'])
elec['GeoName']=elec['GeoName'].str.replace('\s+$','')

In [175]:
elec_state_list=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware",'District Of Columbia',"Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

elec=elec.loc[elec['GeoName'].isin(elec_state_list),:]